In [14]:
from keras.models import Sequential
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense, Activation, Dropout

np.random.seed(42)

train = pd.read_csv('train.csv.zip', compression='zip')
X_train = train.drop(['id', 'species'], axis=1).values
le = LabelEncoder().fit(train['species'])
Y_train = le.transform(train['species'])

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

model = Sequential()
model.add(Dense(1024,input_dim=192, init='uniform', activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(99, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.fit(X_train, Y_train, nb_epoch=32, batch_size=32, validation_split=0.33)

test = pd.read_csv('test.csv.zip', compression='zip')
test_ids = test.pop('id')
X_test = test.values
X_test = scaler.transform(X_test)

Y_test = model.predict_proba(X_test)

submission = pd.DataFrame(Y_test, index=test_ids, columns=le.classes_)
submission.to_csv('submission_nn.csv')

Train on 663 samples, validate on 327 samples
Epoch 1/32
663/663 [==============================] - 0s - loss: 3.8639 - acc: 0.2293 - val_loss: 2.9486 - val_acc: 0.4924
Epoch 2/32
663/663 [==============================] - 0s - loss: 2.2043 - acc: 0.5958 - val_loss: 1.9978 - val_acc: 0.6942
Epoch 3/32
663/663 [==============================] - 0s - loss: 1.4087 - acc: 0.8100 - val_loss: 1.3406 - val_acc: 0.8104
Epoch 4/32
663/663 [==============================] - 0s - loss: 0.8852 - acc: 0.8974 - val_loss: 0.8356 - val_acc: 0.9327
Epoch 5/32
663/663 [==============================] - 0s - loss: 0.5468 - acc: 0.9457 - val_loss: 0.5327 - val_acc: 0.9664
Epoch 6/32
663/663 [==============================] - 0s - loss: 0.3233 - acc: 0.9789 - val_loss: 0.3647 - val_acc: 0.9602
Epoch 7/32
663/663 [==============================] - 0s - loss: 0.2182 - acc: 0.9804 - val_loss: 0.2442 - val_acc: 0.9817
Epoch 8/32
663/663 [==============================] - 0s - loss: 0.1373 - acc: 0.9879 - val_l